In [1]:
import requests
import os 
import pandas as pd

from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
# http authentication scheme with password and user
# Pub/Private key from reddit
auth = requests.auth.HTTPBasicAuth(
    os.environ['CLIENT_ID'],
    os.environ['SECRET_KEY']
)

In [4]:
# Login to reddit
# For getting auth token
data = {
    "grant_type": "password",
    "username": os.environ['name'],
    "password": os.environ['pw'],
}

In [5]:
headers = {
    "User-Agent": "MyAPI/0.0.1"
}

In [6]:
res = requests.post(
    "https://www.reddit.com/api/v1/access_token",
    auth = auth,
    data = data,
    headers = headers
)

In [7]:
TOKEN = res.json()["access_token"]

In [8]:
headers = {**headers, "Authorization": f"bearer {TOKEN}"}

In [9]:
# Stream lastest post
res = requests.get(
    "https://oauth.reddit.com/r/python/new", 
    headers=headers,
    params = {"limit": 100}
)

In [10]:
res.json()["data"]["children"][0]["data"]

{'approved_at_utc': None,
 'subreddit': 'Python',
 'selftext': 'I just published the pre-release 64-bit runtime of [Peaks Consolidation](https://github.com/hkpeaks/peaks-consolidation/releases/tag/v23.05.18) for Windows/Linux on [GitHub](https://github.com/hkpeaks/peaks-consolidation/releases/tag/v23.05.18). I’m so proud of this project because it makes the impossible possible. You can find it here:\n\nAfter three months of intensive exercise, I’m going to take a break now. Without the real and amazing performance of Polars, I would not have had the energy to do this exercise. If you have any problems or comments, please feel free to inform me. This is a very beginnning of the software, there are a lot of room for continuing improvement.',
 'author_fullname': 't2_9k0sxzns1',
 'saved': False,
 'mod_reason_title': None,
 'gilded': 0,
 'clicked': False,
 'title': 'Rome and software weren’t built in a day',
 'link_flair_richtext': [{'e': 'text', 't': 'News'}],
 'subreddit_name_prefixed': '

In [11]:
use_cols = [
    "subreddit",
    "title",
    "selftext",
    "upvote_ratio",
    "ups",
    "downs",
    "score",
    "created_utc",
    "name"
]

def parse_data(data: dict)->dict:
    return {
        col : data[col] for col in use_cols
    }

df = pd.DataFrame(
    [parse_data(row["data"]) for row in res.json()["data"]["children"]]
)

In [12]:
df

,subreddit,title,selftext,upvote_ratio,ups,downs,score,created_utc,name
0,Python,Rome and software weren’t built in a day,I just published the pre-release 64-bit runtim...,1.00,1,0,1,1.684490e+09,t3_13lqw9t
1,Python,Are there any good syntax or style checkers re...,,0.75,4,0,4,1.684484e+09,t3_13low98
2,Python,Propan - is a new python framework for buildin...,Hello everyone!\n\nI have accumulated a critic...,0.87,16,0,16,1.684478e+09,t3_13lncci
3,Python,CSV serialisation and deserialisation,Hey guyss wanted some thoughts/opinion/sugesti...,0.50,0,0,0,1.684475e+09,t3_13lm6g9
4,Python,What if: python without commas,"Since we are having whitespace be meaningful, ...",0.33,0,0,0,1.684474e+09,t3_13lm5cy
...,...,...,...,...,...,...,...,...,...
95,Python,A Python package to retrieve data from Steam,"Hi guys, I recently just build this lightweigh...",0.94,248,0,248,1.684156e+09,t3_13i7oyy
96,Python,Introducing Nautikos; a Python tool for updati...,For my organization I've created a small Pytho...,0.73,5,0,5,1.684155e+09,t3_13i7jn8
97,Python,Implementing SVM and Kernel SVM with Python's ...,,0.55,1,0,1,1.684155e+09,t3_13i76zx
98,Python,"I interviewed Joris Schellekens, author of the...",,0.90,47,0,47,1.684154e+09,t3_13i6wo4
